<a href="https://colab.research.google.com/github/takakou08/GitSdk/blob/master/natureremo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

from urllib.parse import urlencode
from urllib.request import urlopen, Request
from urllib.error import HTTPError
from json import loads
import datetime

api_key = "wq-Cj1UBGmix8ZQu4llv3d4dnoB61UZRek1sk83mMs0.tcslmtYf6QYW3ssnbo9_TyM1ObK7YDkbbSqCyNBd-pk" # APIアクセストークン

url = "https://api.nature.global/1/devices/"

headers = {
    "accept" :"application/json",
    "Authorization" :"Bearer " + api_key,
}

request = Request(url, headers=headers)

try:
  with urlopen(request) as response:
    data_byte = response.read()
    data= loads(data_byte)
except HTTPError as e:
    print(e)

device_info = data[0]["newest_events"]

print("温度 : "   + str(device_info["te"]["val"]) + "度")
print("湿度 : "   + str(device_info["hu"]["val"]) + "%")
print("明るさは : " + str(device_info["il"]["val"]) + "度")

detect_date_str = str(device_info["mo"]["created_at"]).split("T")[0].split("-")
detect_time_str = str(device_info["mo"]["created_at"]).split("T")[1].split("Z")[0].split(":")

detect_date = [int(n) for n in detect_date_str]
detect_time = [int(n) for n in detect_time_str]

date = datetime.datetime(year=detect_date[0], month=detect_date[1], day=detect_date[2], hour=detect_time[0], minute=detect_time[1], second=detect_time[2])
now = datetime.datetime.now()
difference_time = now - date

if difference_time.days > 0:
  print("人感センサー :{}日前".format(difference_time.days))
elif difference_time.total_seconds() > 3600:
  print("人感センサー : {}時間前".format(difference_time.total_seconds/3600))
else:
  print("人感センサー :{}分前".format(int(difference_time.total_seconds()/60)))


温度 : 24.9度
湿度 : 60%
明るさは : 200度
人感センサー :3日前


In [3]:
from urllib.parse import urlencode
from urllib.request import urlopen, Request
from urllib.error import HTTPError
from json import loads

appliance = "TV" #操作する家電の種類["TV", "LIGHT"]
nickname = "研究室TV"   #テレビ/ライトのニックネーム
#nickname = "Atrium-light"   #ライトのニックネーム
api_access_key = "wq-Cj1UBGmix8ZQu4llv3d4dnoB61UZRek1sk83mMs0.tcslmtYf6QYW3ssnbo9_TyM1ObK7YDkbbSqCyNBd-pk" # APIアクセストークン

url = "https://api.nature.global"

headers = {
    "Authorization" :"Bearer " + api_access_key,
    "accept" :"application/json",
    "Content-Type" :"application/x-www-form-urlencoded"
}

#全ての家電情報を取得
request = Request(url + "/1/appliances/", headers=headers)

try:
  with urlopen(request) as response:
    data = response.read()
    devices = loads(data)
except HTTPError as e:
    print(e)


#デバイスID探索
device_id = ""

for device in devices:
  if device["type"] == appliance and device["nickname"] == nickname:
    device_id = device["id"]
    buttons = device[appliance.lower()]["buttons"]


#各ボタンの表示
print("操作するボタンを数字で選んでください。")
num = 0
for button in buttons:
  if button["label"] != "":#空白のボタンがあるためif文で処理
    print(str(num) + " :\t"  + button["label"])
    num += 1

selected_button = int(input("入力:"))
signal = buttons[selected_button]["name"]

#データ送信
request = Request(url + "/1/appliances/" + device_id + "/" + appliance.lower(), headers=headers)

data = {
    "button": signal
}

data = urlencode(data).encode("utf-8")

try:
  urlopen(request, data)
  print("成功です。")
except HTTPError as e:
    print(e)


操作するボタンを数字で選んでください。
0 :	Power
1 :	Source
2 :	Schedule
3 :	Mute
4 :	Terrestrial
5 :	BS
6 :	CS
7 :	Select Audio
8 :	1
9 :	2
10 :	3
11 :	4
12 :	5
13 :	6
14 :	7
15 :	8
16 :	9
17 :	10
18 :	11
19 :	12
20 :	Back
21 :	Menu
22 :	Display Change
23 :	Data
24 :	Next Channel
25 :	Previous Channel
26 :	Left
27 :	Top
28 :	Right
29 :	Bottom
30 :	Ok
31 :	Volume Up
32 :	Volume Down
33 :	Blue
34 :	Red
35 :	Green
36 :	Yellow
37 :	Fast Rewind
38 :	Play
39 :	Fast Forward
40 :	Previous
41 :	Pause
42 :	Stop
43 :	Next
44 :	Clear Sound
45 :	Rec List
46 :	Settings
47 :	Subtitle
48 :	Exit
49 :	Rewind 10sec
50 :	Forward 30sec
入力:0
成功です。


In [2]:
from urllib.parse import urlencode
from urllib.request import urlopen, Request
from urllib.error import HTTPError
from json import loads

temperature = list(range(16, 31)) #温度
operation_mode = ["cool", "warm", "dry", "blow", "auto"] #モード
air_volume = ["1", "2", "3", "4", "5", "auto"] #風量
air_direction = ["1", "2", "3", "4", "5", "auto"] #風向き
button = ["", "power-off"] #電源 空白は電源オン

nickname = "研究室サーバーエアコン"   #エアコンのニックネーム
api_key = "wq-Cj1UBGmix8ZQu4llv3d4dnoB61UZRek1sk83mMs0.tcslmtYf6QYW3ssnbo9_TyM1ObK7YDkbbSqCyNBd-pk" # APIアクセストークン

url = "https://api.nature.global"

headers = {
    "accept" :"application/json",
    "Authorization" :"Bearer " + api_key,
    "Content-Type" :"application/x-www-form-urlencoded"
}

req = Request(url + "/1/appliances/", headers=headers)

try:
  with urlopen(req) as response:
    data = response.read()
    devices = loads(data)
except HTTPError as e:
    print(e)

AirCon_device_id = ""

for device in devices:
  if device["type"] == "AC" and device["nickname"] == nickname:
    AirCon_device_id = device["id"]
    buttons = device["aircon"]

request = Request(url + "/1/appliances/" + AirCon_device_id + "/aircon_settings", headers=headers)

data = {
    "temperature": temperature[10],
    "operation_mode": operation_mode[3],
    "air_volume": air_volume[4],
    "air_direction": air_direction[2],
    "button": button[1],
}

data = urlencode(data).encode("utf-8")

try:
  response = urlopen(request, data)
  print("成功です。")
except HTTPError as e:
    print(e)



成功です。


In [ ]:
from remo import NatureRemoAPI
api = NatureRemoAPI('wq-Cj1UBGmix8ZQu4llv3d4dnoB61UZRek1sk83mMs0.tcslmtYf6QYW3ssnbo9_TyM1ObK7YDkbbSqCyNBd-pk')
devices = api.get_devices()
devices